In [2]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 250

In [3]:
sys.path.append('../')
import read_player_stats
import training_data

In [4]:
qb_game_df = pd.read_csv('../game_stats/qb_game_stats.csv',)

In [9]:
qb_game_df = qb_game_df.drop(['Unnamed: 0'], axis = 1)
qb_game_df

,Name,Team,Games,PassComp,PassAtt,PassYards,PassTD,INT,RunAtt,RunYards,RunTD,FFP,FFPPG,Season,Week
0,AJFeeley,MIA,1,21,31,168,1,1,0,0,0,12.4,12.4,2004,1
1,AJFeeley,MIA,1,21,39,218,1,2,1,3,0,15.2,15.2,2004,2
2,AJFeeley,MIA,1,13,27,137,0,2,4,1,0,7.0,7.0,2004,3
3,AJFeeley,MIA,1,1,2,6,0,0,0,0,0,0.3,0.3,2004,5
4,AJFeeley,MIA,1,6,15,129,0,1,0,0,0,6.5,6.5,2004,9
5,AJFeeley,MIA,1,23,45,229,1,2,1,7,1,22.2,22.2,2004,11
6,AJFeeley,MIA,1,17,33,159,2,1,4,2,0,16.2,16.2,2004,12
7,AJFeeley,MIA,1,25,51,303,3,5,0,0,0,27.2,27.2,2004,13
8,AJFeeley,MIA,1,17,35,170,1,1,2,2,0,12.7,12.7,2004,14
9,AJFeeley,MIA,1,22,35,198,1,0,2,-2,0,13.7,13.7,2004,15


In [10]:
def game_dist(entry1, entry2):
    return (entry1['Season'] - entry2['Season'], entry1['Week'] - entry2['Week'])

In [35]:
qb_game_df['Name'].value_counts()

DrewBrees            174
EliManning           169
TomBrady             161
BenRoethlisberger    160
PeytonManning        160
TonyRomo             151
PhilipRivers         149
CarsonPalmer         146
MattSchaub           130
MattHasselbeck       124
JayCutler            119
JoeFlacco            112
AlexSmith            111
AaronRodgers         110
MattRyan             110
BrettFavre           109
MichaelVick          109
BradSmith            106
RyanFitzpatrick       97
DonovanMcNabb         97
MattCassel            91
JasonCampbell         91
KyleOrton             86
JakeDelhomme          81
DavidGarrard          81
MatthewStafford       77
KerryCollins          76
MarcBulger            74
MarkBrunell           73
KurtWarner            73
MarkSanchez           71
DavidCarr             70
BradJohnson           66
SenecaWallace         65
ChadHenne             64
AndyDalton            64
DerekAnderson         63
CamNewton             62
VinceYoung            62
ChadPennington        61


In [37]:
keep = qb_game_df['Name'].value_counts() > 20
keep

DrewBrees             True
EliManning            True
TomBrady              True
BenRoethlisberger     True
PeytonManning         True
TonyRomo              True
PhilipRivers          True
CarsonPalmer          True
MattSchaub            True
MattHasselbeck        True
JayCutler             True
JoeFlacco             True
AlexSmith             True
AaronRodgers          True
MattRyan              True
BrettFavre            True
MichaelVick           True
BradSmith             True
RyanFitzpatrick       True
DonovanMcNabb         True
MattCassel            True
JasonCampbell         True
KyleOrton             True
JakeDelhomme          True
DavidGarrard          True
MatthewStafford       True
KerryCollins          True
MarcBulger            True
MarkBrunell           True
KurtWarner            True
MarkSanchez           True
DavidCarr             True
BradJohnson           True
SenecaWallace         True
ChadHenne             True
AndyDalton            True
DerekAnderson         True
C